In [ ]:
from google.colab import drive
drive.mount('/data/')
data_dir = '/data/My Drive/Colab Notebooks/Twitter'
!ls '/data/My Drive/Colab Notebooks/Twitter'
!pip install matplotlib

Drive already mounted at /data/; to attempt to forcibly remount, call drive.mount("/data/", force_remount=True).
'Moss (1).gsheet'
 Moss.csv
 Moss.gsheet
'Moss on Stone 20211108.xlsx'
'Tweets by Moss on Stone 20211026 Early Newspapers and Magazines.xlsx'
'Tweets for Digital collection project 20210526.xlsx'


In [ ]:
import pandas as pd
import os
import requests
import re
from collections import Counter
import numpy as np

import nltk
from nltk.stem import *
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_excel(data_dir+'/Moss on Stone 20211108.xlsx')
df.head(2)

,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,media,urls,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_or_quote_id,retweet_or_quote_screen_name,retweet_or_quote_user_id,source,user_id,user_created_at,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,1450839702710407174,https://twitter.com/MossonStone1/status/145083...,Wed Oct 20 15:01:55 +0000 2021,2021-10-20T15:01:55+00:00,MossonStone1,湘學報 1897-1898 - 7 issues of this late Qing dyn...,original,NaN,primarysources chinesehistory,https://pbs.twimg.com/media/FCJqzw4XIAIiJgZ.jpg,https://archive.org/search.php?query=title%3A%...,3,NaN,NaN,NaN,en,NaN,0.0,1,NaN,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1433339407399063552,Thu Sep 02 08:02:08 +0000 2021,False,Love East Asian history. A big fan of the inte...,296,538,604,3,Incheon,Moss on Stone,145,NaN,https://archive.org/details/@g_y_library,False
1,1450829228753268738,https://twitter.com/MossonStone1/status/145082...,Wed Oct 20 14:20:17 +0000 2021,2021-10-20T14:20:17+00:00,MossonStone1,📢 We've completed a ten-year project to digiti...,retweet,NaN,NaN,NaN,NaN,343,NaN,NaN,NaN,en,NaN,NaN,186,1.442772e+18,bodleianlibs,135441822.0,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1433339407399063552,Thu Sep 02 08:02:08 +0000 2021,False,Love East Asian history. A big fan of the inte...,296,538,604,3,Incheon,Moss on Stone,145,NaN,https://archive.org/details/@g_y_library,False


In [ ]:
data = df[df['tweet_type']=="original"]
print(len(data))
data.reset_index(drop=True, inplace=True)

116


In [ ]:
data.head(2)

,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,media,urls,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_or_quote_id,retweet_or_quote_screen_name,retweet_or_quote_user_id,source,user_id,user_created_at,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,1450839702710407174,https://twitter.com/MossonStone1/status/145083...,Wed Oct 20 15:01:55 +0000 2021,2021-10-20T15:01:55+00:00,MossonStone1,湘學報 1897-1898 - 7 issues of this late Qing dyn...,original,NaN,primarysources chinesehistory,https://pbs.twimg.com/media/FCJqzw4XIAIiJgZ.jpg,https://archive.org/search.php?query=title%3A%...,3,NaN,NaN,NaN,en,NaN,0.0,1,NaN,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1433339407399063552,Thu Sep 02 08:02:08 +0000 2021,False,Love East Asian history. A big fan of the inte...,296,538,604,3,Incheon,Moss on Stone,145,NaN,https://archive.org/details/@g_y_library,False
1,1450763229068660736,https://twitter.com/MossonStone1/status/145076...,Wed Oct 20 09:58:02 +0000 2021,2021-10-20T09:58:02+00:00,MossonStone1,"民國叢書 - Some 1,273 volumes of books published d...",original,NaN,primarysources chinesehistory,NaN,https://archive.org/search.php?query=title%3A%...,14,NaN,NaN,NaN,ja,NaN,0.0,11,NaN,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1433339407399063552,Thu Sep 02 08:02:08 +0000 2021,False,Love East Asian history. A big fan of the inte...,296,538,604,3,Incheon,Moss on Stone,145,NaN,https://archive.org/details/@g_y_library,False


In [ ]:
def comma(s):
    return(s.replace(',', ''))

def chinese(line):
    for n in re.findall(r'[\u4e00-\u9fff]+', line):
        return n

def start_year(SearchMe):

    find_tx=SearchMe.find("-")
    sec_tx=SearchMe.find("-", find_tx+1)
    third_tx=SearchMe.find("-", sec_tx+1)
    if find_tx == -1:
        return 0
    else:  
        if (SearchMe[find_tx-3:find_tx].isdigit() & SearchMe[find_tx+1:find_tx+4].isdigit()):
            start=SearchMe[:find_tx].split(" ",-1)[-1]
            if len(start)>4:
              return(int(start[-4:]))
            else:
              return(int(start))
        elif (SearchMe[sec_tx-3:sec_tx].isdigit() & SearchMe[sec_tx+1:sec_tx+4].isdigit()):
            start=SearchMe[:sec_tx].split(" ",-1)[-1]
            if len(start)>4:
              return(int(start[-4:]))
            else:
              return(int(start))
        elif (SearchMe[third_tx-3:third_tx].isdigit() & SearchMe[third_tx+1:third_tx+4].isdigit()):
            start=SearchMe[:third_tx].split(" ",-1)[-1]
            if len(start)>4:
              return(int(start[-4:]))
            else:
              return(int(start))
        else:
            return 0
    
def end_year(SearchMe):

    find_tx=SearchMe.find("-")
    sec_tx=SearchMe.find("-", find_tx+1)
    third_tx=SearchMe.find("-", sec_tx+1)
    if find_tx == -1:
        return 0
    
    else:  
        if (SearchMe[find_tx-3:find_tx].isdigit() & SearchMe[find_tx+1:find_tx+4].isdigit()):
            end=SearchMe[find_tx+1:].split(" ",1)[0]
            if len(end)>4:
              return(int(end[:-1]))
            else:
              return(int(end))
        elif (SearchMe[sec_tx-3:sec_tx].isdigit() & SearchMe[sec_tx+1:sec_tx+4].isdigit()):  
            end=SearchMe[sec_tx+1:].split(" ",1)[0]
            if len(end)>4:
              return(int(end[:-1]))
            else:
              return(int(end))
        elif (SearchMe[third_tx-3:third_tx].isdigit() & SearchMe[third_tx+1:third_tx+4].isdigit()):
            end=SearchMe[third_tx+1:].split(" ",1)[0]
            if len(end)>4:
              return(int(end[:-1]))
            else:
              return(int(end))
        else:
            return 0

def find_issues(line):
    ct_issues=line.count('issues')
    ct_volumes=line.count('volumes')
    ct_over=line.lower().count('over')
    if ct_issues!=0:
        fnl=re.findall(r'(\d+) issues', line)
        for p in fnl: return(int(p))
    elif ct_volumes!=0:
        fnl=re.findall(r'(\d+) volumes', line)
        for p in fnl: return(int(p))
    elif ct_over!=0:
        find_over = line.lower().find('over')
        new_cover=line[find_over+5:].split(" ",1)[0] 
        return (int(new_cover))
    else:
        return 0

In [ ]:
data['text'] = data['text'].apply(lambda d: comma(d))
data['Chinese'] = data['text'].apply(lambda d: chinese(d))
data['start'] = data['text'].apply(lambda d: start_year(d))
data['end'] = data['text'].apply(lambda d: end_year(d))
data['NumberOFDoc'] = data['text'].apply(lambda d: find_issues(d))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
data['NumberOFDoc']=data['NumberOFDoc'].fillna(0).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data[['text','Chinese','start','end','NumberOFDoc',"urls"]].values

array([['湘學報 1897-1898 - 7 issues of this late Qing dynasty reformist journal published in Changsha Hunan on Internet Archive with issue PDF downloads. #primarysources #chinesehistory https://t.co/YdCg8HQ0Nf https://t.co/SjsSuE5gbL',
        '湘學報', 1897, 1898, 7,
        'https://archive.org/search.php?query=title%3A%28%E6%B9%98%E5%AD%B8%E5%A0%B1%29+AND+description%3A%28%E6%B9%98%E5%AD%B8%E5%A0%B1%29&sort=date'],
       ['民國叢書 - Some 1273 volumes of books published during the 民國 period 1912-1949 are on Internet Archive from the 民國叢書 reprints. 1/2 #primarysources #chinesehistory https://t.co/gIgOPS3jZ9',
        '民國叢書', 1912, 1949, 1273,
        'https://archive.org/search.php?query=title%3A%28%22%E6%B0%91%E5%9B%BD%E4%B8%9B%E4%B9%A6%22%29%20AND%20mediatype%3A%28texts%29%20AND%20subject%3A%28%22Republic%20of%20China%22%29'],
       ["國聞報 1897-1899 - Over 400 issues of Yan Fu's Guowenbao on Internet Archive with issue PDF downloads. However the last 150 issues have a bizarre flaw in digit

### NLP Text Analysis

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def process(line):
    find_link=line.find("https:")
    new_text=line[:find_link]
    return([wordnet_lemmatizer.lemmatize(t) for t in tokenizer.tokenize(new_text) if ((t not in stop_words) & (t.isnumeric()==False))])
    
tokenizer = RegexpTokenizer(r'\w+')
data['tokens']=data['text'].str.lower().apply(process)
data['tokens']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0     [湘學報, issue, late, qing, dynasty, reformist, j...
1     [民國叢書, volume, book, published, 民國, period, in...
2     [國聞報, issue, yan, fu, guowenbao, internet, arc...
3     [民報, issue, important, minbao, magazine, inter...
4     [南京日報, issue, nanjing, daily, internet, archiv...
                            ...                        
92    [新天津畫報, issue, new, tianjin, pictorial, intern...
93    [新天津畫報, issue, new, tianjin, pictorial, intern...
94    [textfiles, thanks, much, setting, recent, col...
95    [庸報, issue, major, tianjin, newspaper, japanes...
96    [新天津畫報, issue, new, tianjin, pictorial, intern...
Name: tokens, Length: 97, dtype: object

In [ ]:
data['tokens'][0]

['湘學報',
 'issue',
 'late',
 'qing',
 'dynasty',
 'reformist',
 'journal',
 'published',
 'changsha',
 'hunan',
 'internet',
 'archive',
 'issue',
 'pdf',
 'downloads',
 'primarysources',
 'chinesehistory']

In [ ]:
data[['text','Chinese','start','end','NumberOFDoc',"urls"]].to_csv(data_dir+'/Moss_20211108.csv')